In [19]:
from keras.layers import *
from keras.models import Sequential, Model
from keras.preprocessing import image

In [4]:
from keras.applications.vgg16 import VGG16

In [30]:
model = VGG16(include_top=False, weights="imagenet", input_shape=(224,224,3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [33]:
for i in range(len(model.layers)):
    print(i, model.layers[i])

0 <keras.engine.input_layer.InputLayer object at 0x0000015D81C49630>
1 <keras.layers.convolutional.Conv2D object at 0x0000015D81C49D30>
2 <keras.layers.convolutional.Conv2D object at 0x0000015D81C48C50>
3 <keras.layers.pooling.MaxPooling2D object at 0x0000015D81C40CF8>
4 <keras.layers.convolutional.Conv2D object at 0x0000015D81C432B0>
5 <keras.layers.convolutional.Conv2D object at 0x0000015D81C69F98>
6 <keras.layers.pooling.MaxPooling2D object at 0x0000015D81C572E8>
7 <keras.layers.convolutional.Conv2D object at 0x0000015D81C53278>
8 <keras.layers.convolutional.Conv2D object at 0x0000015D80D62FD0>
9 <keras.layers.convolutional.Conv2D object at 0x0000015D81A19780>
10 <keras.layers.pooling.MaxPooling2D object at 0x0000015D836225C0>
11 <keras.layers.convolutional.Conv2D object at 0x0000015D836229E8>
12 <keras.layers.convolutional.Conv2D object at 0x0000015D83631E10>
13 <keras.layers.convolutional.Conv2D object at 0x0000015D8365D898>
14 <keras.layers.pooling.MaxPooling2D object at 0x000001

In [35]:
for i in range(16):
    model.layers[i].trainable = False

In [36]:
from keras.optimizers import Adam

In [37]:
model.compile(Adam(lr = 0.0001), loss='categorical_crossentropy')

In [38]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

## Build Your Classifier Keras Functional API

In [39]:
x = model.output
# model.add(GlobalAveragePooling2D())
gap = GlobalAveragePooling2D()(x)
d1 = Dense(256, activation="relu")(gap)
out = Dense(4, activation="softmax")(d1)

new_model = Model(inputs = model.input, outputs = out)

In [40]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [17]:
new_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ["accuracy"])

##  Fit Model

In [20]:
train_data = image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True,
        width_shift_range=0.2,
        horizontal_flip=True)
val_data = image.ImageDataGenerator(rescale=1.0/255.)

In [25]:
train_generator  = train_data.flow_from_directory("images/",
                                                  target_size=(224,224),
                                                 batch_size=32,
                                                class_mode="categorical",
                                                 shuffle=True)

Found 724 images belonging to 4 classes.


In [26]:
val_generator  = train_data.flow_from_directory("val_images/",
                                                  target_size=(224,224),
                                                 batch_size=32,
                                                class_mode="categorical",
                                                 shuffle=True)

Found 84 images belonging to 4 classes.


In [27]:
hist = new_model.fit_generator(train_generator, steps_per_epoch=25,
                               epochs=20,
                           validation_data=val_generator,
                           validation_steps=3)

Epoch 1/20
25/25 [==============================] - 328s 13s/step - loss: 1.2170 - val_loss: 0.9638
Epoch 2/20
 7/25 [=======>......................] - ETA: 3:29 - loss: 0.9672

KeyboardInterrupt: 

In [29]:
new_model.evaluate_generator(val_generator, steps=3)

0.9481270086197626